<a href="https://colab.research.google.com/github/Onmang/Multimedia_engineering/blob/master/T20ME022_mid_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

中間レポート
T20ME022 グエンコンフィ

In [21]:
!unzip "/content/resistor_v3.zip"

Archive:  /content/resistor_v3.zip
replace data/valid/1600/1600_230_0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
# ライブラリのインポート
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os # ディレクトリ（フォルダ）やファイルを扱うためのライブラリ（本当はPathlibライブラリのほうが良いが難しいので簡単な方で）
import glob # ファイル一覧を取得するためのライブラリ
import re # 正規表現を使ったパターンマッチング用（ラベルを取得するため）
from keras.preprocessing import image # keras.preprocessing.image APIを利用する。画像拡張用の関数が用意されている。
from keras.preprocessing.image import ImageDataGenerator
tf.test.gpu_device_name() # GPUの利用確認

'/device:GPU:0'

In [2]:
train_list = glob.glob('data/train/*/*.png') # 訓練用画像ファイルの取得（拡張子がpng）
valid_list = glob.glob('data/valid/*/*.png') # 検証用画像ファイルの取得
classes = sorted(os.listdir('data/train'), key=int) # 教師ラベルの一覧をリストで取得する。"sorted"でソートしておく。
print(classes) # 取得した抵抗器のラベルを表示。文字列になっている点に注意すること！

['150', '160', '390', '430', '620', '1600', '1800', '2200', '2400', '3000', '3300', '3600', '5600', '9100']


In [3]:
# 画像変換するためのクラスをセットアップ
# あまりドラスティックに変えるとはみ出るのでそこそこにする
# ImageDataGeneratorを利用
generator = ImageDataGenerator(
            rotation_range=90,       # 画像の回転
            width_shift_range=0.1,    # 横方向シフト （3通りの方法あり）
            height_shift_range=0.1,    # 縦方向シフト
            zoom_range=[0.8, 1.0],     # ズーム
            horizontal_flip=True,   # 横方向のフラップ（折返し）
            vertical_flip=True,      # 縦方向のフラップ
            fill_mode='nearest',
            )

In [ ]:
# データ拡張処理と保存
# 拡張したデータはオリジナル画像と同じフォルダに保存されるが，拡張子は jpg にしてある（区別するため）
# Albumentationを使う場合はソースコードを修正する必要がある。
for file in train_list:
    dir = os.path.dirname(file) # ファイルパス（ファイルが保存されているフォルダ）を取得
    base = os.path.basename(file).split('.')[0] # ファイル名を取得（拡張子なし）
    max_img_num = 5 # 1つの画像あたり10倍に拡張
    img = cv2.imread(file) # 画像を読み込み
    img = img[np.newaxis,:] # 4次元テンソルに変換
    id = 0
    for d in generator.flow(img, batch_size=1): # flow関数で画像をランダムに変換する
        ofile = f'{base}_A{id:03d}.jpg' # 生成した画像はjpgで保存
        output = os.path.join(dir, ofile) # 保存するファイル名を設定（ディレクトリ情報付き）
        cv2.imwrite(output, d[0, :])  # ファイルに保存
        print(f'{output} saved.')
        id += 1
        # flow関数は無限ループするので必要な枚数生成できたらループを抜ける
        if (id % max_img_num) == 0:
            break

In [12]:
train_ds = tf.data.Dataset.list_files(train_list) # 訓練ファイル名のリストをTensor型に変換
valid_ds = tf.data.Dataset.list_files(valid_list) # 検証用のファイル名のリストをTensor型に変換

# ファイル名から画像データをロードしてNNへ入力できるようにデータを成形する。ついでに教師ラベルも取得する
def load_file(files):
    ys = [] # ラベル
    xs = [] # 入力データ
    for f in files:
        file = bytes.decode(f.numpy()) # ファイル名はTensor型で保存されているため，文字列型として取得する。
        m = re.search(r'/(\d+)_', file) # 正規表現を使ってファイル名から抵抗値を取得する。
        label = m.groups()[0] # 抵抗値を取得しlabelに保存
        ys.append(label) # ラベルをラベルリストに追加する
        img = cv2.imread(file) # 画像データをカラーで取得。画像サイズは64x64になっているのでここではリサイズしない。
        xs.append(img) # データを入力データリストに追加
    xs = np.array(xs, dtype=np.float32) / 255. # 正規化してfloat32の行列に変換する
    ys = np.array(ys, dtype=np.float32) # ラベルも行列に変換
    return xs, ys

#
# tf.Dataによるtf.Tensor変換
#
AUTOTUNE = tf.data.experimental.AUTOTUNE # 処理を最適化するためのおまじない（自動チューニング設定）
train_ds = train_ds.shuffle(len(train_list)) # 訓練データをシャッフルする。引数にはデータ数を指定すると完全なシャッフルが行われる。len(x_train)は60000。
train_ds = train_ds.repeat(1) # 1 epochで使われるデータの回数。1の場合，1epochで1回しか使われない。引数を空欄にすると無限に使われる。
train_ds = train_ds.batch(64) # ミニバッチを作る。1バッチ32個のデータ。
train_ds = train_ds.map(lambda files: tf.py_function(load_file, [files], Tout=[tf.float32, tf.float32])) # ファイル名から入力ラベルとラベルを取得
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE) # 訓練中に次のバッチを取り出すための処理。

valid_ds = valid_ds.batch(64) # 検証データはシャッフルする必要ないので，バッチ化のみの処理でOK。
valid_ds = valid_ds.map(lambda x: tf.py_function(load_file, [x], Tout=[tf.float32, tf.float32]))

In [15]:
# Functional API
input = Input(shape=(64, 64, 3), name='input') # 入力層の定義　64×64×3 （カラー画像）
h = Conv2D(16, (3, 3), padding='same', activation='relu', name='cnn01')(input)
h = MaxPooling2D((2, 2), name='pool01')(h)
h = Conv2D(32, (3, 3), padding='same', activation='relu', name='cnn02')(h)
h = MaxPooling2D((2, 2), name='pool02')(h)
h = Dropout(rate=0.2)(h)
h = Conv2D(64, (3, 3), padding='same', activation='relu', name='cnn03')(h)
h = MaxPooling2D((2, 2), name='pool03')(h)
h = Dropout(rate=0.2)(h)
h = Conv2D(128, (3, 3), padding='same', activation='relu', name='cnn04')(h)
h = MaxPooling2D((2, 2), name='pool04')(h)
h = Dropout(rate=0.2)(h)
h = Conv2D(256, (3, 3), padding='valid', activation='relu', name='cnn05')(h)
h = MaxPooling2D((2, 2), name='pool05')(h)
h = Flatten(name='flatten')(h) # GlobalAveragePoolingでも良い
h = Dense(128, activation='relu', name='dense01')(h) # 全結合層の隠れ層のノードは128
output = Dense(1, activation='linear', name='output')(h) # 出力層

model = Model(inputs=input, outputs=output) # この処理でモデルを実体化する。入力層と出力層を渡すと自動的にネットワークを構築してくれる。
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 cnn01 (Conv2D)              (None, 64, 64, 16)        448       
                                                                 
 pool01 (MaxPooling2D)       (None, 32, 32, 16)        0         
                                                                 
 cnn02 (Conv2D)              (None, 32, 32, 32)        4640      
                                                                 
 pool02 (MaxPooling2D)       (None, 16, 16, 32)        0         
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 cnn03 (Conv2D)              (None, 16, 16, 64)        1849

In [16]:
# TFのバグでこのように書く

#model.compile(optimizer='adam', loss='MSE', metrics=['MAE'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), loss='MSE', metrics=['MAE'])
# 訓練の実施
model.fit(train_ds, epochs=40, validation_data=valid_ds)

Epoch 1/40
52/52 [==============================] - 7s 73ms/step - loss: 5323212.0000 - MAE: 1679.2743 - val_loss: 5083201.0000 - val_MAE: 1531.8256
Epoch 2/40
52/52 [==============================] - 3s 60ms/step - loss: 4452913.5000 - MAE: 1551.0417 - val_loss: 5347021.0000 - val_MAE: 1385.6010
Epoch 3/40
52/52 [==============================] - 3s 60ms/step - loss: 4486718.0000 - MAE: 1551.1445 - val_loss: 5066563.0000 - val_MAE: 1807.0126
Epoch 4/40
52/52 [==============================] - 3s 66ms/step - loss: 4565300.0000 - MAE: 1577.3650 - val_loss: 5028147.5000 - val_MAE: 1421.8251
Epoch 5/40
52/52 [==============================] - 3s 61ms/step - loss: 4403099.0000 - MAE: 1541.4844 - val_loss: 4931529.5000 - val_MAE: 1528.2721
Epoch 6/40
52/52 [==============================] - 3s 58ms/step - loss: 4379899.0000 - MAE: 1542.2500 - val_loss: 4887632.5000 - val_MAE: 1361.6146
Epoch 7/40
52/52 [==============================] - 4s 71ms/step - loss: 4276384.0000 - MAE: 1517.3660 - v

KeyboardInterrupt: ignored

In [8]:
# 評価
errors = []
for file in glob.glob('data/test/*.png'):
    m = re.search(r'/(\d+)_', file) # ちょっと違うパターンの書き方
    label = float( m.groups()[0] ) # 実数に変換
    img = cv2.imread(file)
    img = img.reshape(1, 64, 64, 3)
    img = np.float32(img) / 255.
    pred = model.predict(img)
    estimate = pred[0][0]
    error = abs(label - estimate) / label * 100
    print(f'Label {label}, Estimate {estimate}, Error Rate: {error}')
    errors.append(error)
ave = np.average(np.array(errors))
print(f'平均誤り率 {ave:.2f}%')

1/1 [==============================] - 0s 113ms/step
Label 2400.0, Estimate 2840.9033203125, Error Rate: 18.3709716796875
1/1 [==============================] - 0s 20ms/step
Label 5600.0, Estimate 5821.91015625, Error Rate: 3.962681361607143
1/1 [==============================] - 0s 22ms/step
Label 2200.0, Estimate 2343.614013671875, Error Rate: 6.527909712357954
1/1 [==============================] - 0s 20ms/step
Label 390.0, Estimate 339.3863830566406, Error Rate: 12.977850498297276
1/1 [==============================] - 0s 22ms/step
Label 1800.0, Estimate 1945.4219970703125, Error Rate: 8.078999837239584
1/1 [==============================] - 0s 23ms/step
Label 430.0, Estimate 540.4139404296875, Error Rate: 25.677660565043603
1/1 [==============================] - 0s 23ms/step
Label 3300.0, Estimate 3116.620361328125, Error Rate: 5.556958747632576
1/1 [==============================] - 0s 20ms/step
Label 3000.0, Estimate 2803.95849609375, Error Rate: 6.534716796874999
1/1 [=========